In [94]:
import numpy as np
import pandas as pd
import pymongo
import datetime
client = pymongo.MongoClient()
db1 = client['Index']
twa = pd.DataFrame(db1['TWA00'].find({'日期': {'$gte': datetime.datetime(2008, 1, 1)}}, {'_id': 0})).set_index('日期')
vix = pd.DataFrame(db1['VIX'].find({'日期': {'$gte': datetime.datetime(2008, 1, 1)}}, {'_id': 0})).set_index('日期')
how_many_day = 1 # 隔幾天的漲跌
today_close = 14532
rate_ = []
std_ = []
pf_ = []
for d in range(1, 10):
    jump = (twa['收盤價'].shift(how_many_day) / twa['收盤價'].shift(how_many_day+d))-1
    rate = today_close / twa['收盤價'].iloc[-d] - 1
    if rate >=0:
        idx = jump[jump >= rate].index
    else:
        idx = jump[jump < rate].index
    next_day = (twa['收盤價'] / twa['收盤價'].shift(how_many_day) - 1).loc[idx]
    rate_.append((next_day>0).sum()/next_day.count()*100)
    std_.append(next_day.std()*100)
    pf_.append(next_day.mean()*100)
    print('近 {} 日 漲跌 {:.2f}% 後 隔{}日漲跌幅平均: {:.2f}% 上漲機率: {:.2f}% 標準差: {:.2f}%'.format(d, rate*100, how_many_day, next_day.mean()*100, (next_day>0).sum()/next_day.count()*100, next_day.std()*100))
print('平均 {} 日後漲跌 {:.2f}% 平均上漲機率 {:.2f}% 平均標準差 {:.2f}%'.format(how_many_day, np.mean(pf_), np.mean(rate_), np.mean(std_)))

近 1 日 漲跌 2.52% 後 隔1日漲跌幅平均: 0.27% 上漲機率: 58.33% 標準差: 1.61%
近 2 日 漲跌 3.74% 後 隔1日漲跌幅平均: 0.42% 上漲機率: 63.46% 標準差: 1.92%
近 3 日 漲跌 7.61% 後 隔1日漲跌幅平均: 0.50% 上漲機率: 50.00% 標準差: 1.30%
近 4 日 漲跌 6.55% 後 隔1日漲跌幅平均: 0.08% 上漲機率: 48.00% 標準差: 1.37%
近 5 日 漲跌 8.87% 後 隔1日漲跌幅平均: -0.58% 上漲機率: 42.86% 標準差: 2.09%
近 6 日 漲跌 9.89% 後 隔1日漲跌幅平均: -1.59% 上漲機率: 37.50% 標準差: 2.41%
近 7 日 漲跌 11.56% 後 隔1日漲跌幅平均: -1.54% 上漲機率: 60.00% 標準差: 2.83%
近 8 日 漲跌 11.90% 後 隔1日漲跌幅平均: -0.48% 上漲機率: 66.67% 標準差: 2.38%
近 9 日 漲跌 10.93% 後 隔1日漲跌幅平均: 0.30% 上漲機率: 78.57% 標準差: 1.35%
平均 1 日後漲跌 -0.29% 平均上漲機率 56.15% 平均標準差 1.92%


In [113]:
how_many_day = 1 # 隔幾天的漲跌
today_close = 14532
rate_ = []
std_ = []
pf_ = []
twa.loc[datetime.datetime.today().strftime('%Y-%m-%d'), '收盤價'] = 14546.31
vix.loc[datetime.datetime.today().strftime('%Y-%m-%d'), '收盤價'] = 24.53
cor = vix['收盤價'].pct_change().rolling(3).corr(twa['收盤價'].pct_change())
for d in range(2, 11):
    cor = vix['收盤價'].pct_change().rolling(d).corr(twa['收盤價'].pct_change())
    a = ((twa['收盤價'] / twa['收盤價'].shift(how_many_day) - 1) > 0)
    a_ = a.sum()/a.count()
    rate = cor.iloc[-1]
    idx = cor[(cor > rate - .5)&(cor <= rate + .5)].index
    next_day = (twa['收盤價'] / twa['收盤價'].shift(how_many_day) - 1).loc[idx]
    rate_.append((next_day>0).sum()/next_day.count()*100)
    std_.append(next_day.std()*100)
    pf_.append(next_day.mean()*100)
    print('cor介於 {:.2f} ~ {:.2f} 隔{}日漲跌幅平均: {:.2f}% 上漲機率: {:.2f}% 標準差: {:.2f}% 樣本上漲機率: {:.2f}%'.format(rate-.2, rate+.2, how_many_day, next_day.mean()*100, (next_day>0).sum()/next_day.count()*100, next_day.std()*100, a_))
print('平均 {} 日後漲跌 {:.2f}% 平均上漲機率 {:.2f}% 平均標準差 {:.2f}%'.format(1, np.mean(pf_), np.mean(rate_), np.mean(std_)))

cor介於 0.80 ~ 1.20 隔1日漲跌幅平均: 0.10% 上漲機率: 55.09% 標準差: 1.06% 樣本上漲機率: 0.54%
cor介於 -0.63 ~ -0.23 隔1日漲跌幅平均: 0.07% 上漲機率: 54.32% 標準差: 1.10% 樣本上漲機率: 0.54%
cor介於 -0.09 ~ 0.31 隔1日漲跌幅平均: 0.08% 上漲機率: 55.65% 標準差: 1.01% 樣本上漲機率: 0.54%
cor介於 -0.15 ~ 0.25 隔1日漲跌幅平均: 0.07% 上漲機率: 55.28% 標準差: 1.04% 樣本上漲機率: 0.54%
cor介於 -0.03 ~ 0.37 隔1日漲跌幅平均: 0.10% 上漲機率: 56.29% 標準差: 1.08% 樣本上漲機率: 0.54%
cor介於 -0.03 ~ 0.37 隔1日漲跌幅平均: 0.08% 上漲機率: 54.66% 標準差: 1.11% 樣本上漲機率: 0.54%
cor介於 0.05 ~ 0.45 隔1日漲跌幅平均: 0.05% 上漲機率: 52.11% 標準差: 1.18% 樣本上漲機率: 0.54%
cor介於 0.19 ~ 0.59 隔1日漲跌幅平均: 0.07% 上漲機率: 52.49% 標準差: 1.32% 樣本上漲機率: 0.54%
cor介於 0.19 ~ 0.59 隔1日漲跌幅平均: 0.04% 上漲機率: 51.08% 標準差: 1.34% 樣本上漲機率: 0.54%
平均 1 日後漲跌 0.07% 平均上漲機率 54.11% 平均標準差 1.14%


In [52]:
a = ((twa['收盤價'] / twa['收盤價'].shift(how_many_day) - 1) > 0)
a.sum()/a.count()

0.569012547735952

In [110]:
((cor > rate - .2)&(cor <= rate + .2)).sum()

164

In [114]:
idx

DatetimeIndex(['2008-01-16', '2008-01-17', '2008-01-18', '2008-01-21',
               '2008-01-22', '2008-01-23', '2008-01-24', '2008-01-25',
               '2008-01-28', '2008-02-14',
               ...
               '2021-11-16', '2021-11-17', '2021-11-18', '2021-11-19',
               '2022-01-04', '2022-01-05', '2022-11-09', '2022-11-11',
               '2022-11-14', '2022-11-15'],
              dtype='datetime64[ns]', name='日期', length=462, freq=None)